In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'/home/lucky/Documents/projects/MyProject/smoke_detection'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
        )

        return prepare_base_model_config

### Component -> base model

In [8]:
import tensorflow as tf

2023-12-30 20:41:10.750404: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 20:41:10.786481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 20:41:10.786513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 20:41:10.787543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 20:41:10.793046: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 20:41:10.793685: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [23]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.full_model = None
        self.model = None
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, learning_rate, IMAGE_SIZE):
        model.trainable = False

        input_ = tf.keras.layers.Input(shape = tuple(IMAGE_SIZE) + (3,), name = "input layer")
        x = model(input_)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        
        output = tf.keras.layers.Dense(2,activation = "sigmoid")(x)
        
        base_model = tf.keras.Model(input_,output)
        base_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate), metrics = ["accuracy"])
        base_model.summary()
        return base_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            learning_rate=self.config.params_learning_rate,
            IMAGE_SIZE = self.config.params_image_size
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [24]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-12-30 20:44:59,925: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-30 20:44:59,928: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 20:44:59,929: INFO: common: created directory at: artifacts]
[2023-12-30 20:44:59,930: INFO: common: created directory at: artifacts/prepare_base_model]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                   